# Executing AWS CloudWatch Queries

In [2]:
import boto3
import subprocess
import json
import os
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

import sys
sys.path.append('..')
from awsutils import *

## Step 1: Create session and client

In [3]:
# Use the function with your profile to get a session
aws_session = set_aws_credentials('acl-production', 'eu-central-1')

# Create clients using the session
sts_client = aws_session.client('sts')
logs_client = aws_session.client('logs')

# Example usage of the clients
account_id = sts_client.get_caller_identity()["Account"]
print("Current AWS Account ID:", account_id)

Current AWS Account ID: 707785685172


## Collect the request ids

In [5]:
def collect_request_ids(data):
    request_ids = []

    # Iterate over the entries and extract the request_id and x_request_id
    for entry in data:
        message = entry["@message"]
        x_request_id = message.get("x_request_id", None)
        x_nginx_id = message.get("x_nginx_id", None)

        if x_request_id or x_nginx_id:
            if x_request_id:
                request_ids.append(x_request_id)
            if x_nginx_id != x_request_id:
                request_ids.append(x_nginx_id)

    return request_ids


## Collect slow queries and corresponding request ids

In [7]:
log_group = 'projects-main'
#log_group =  'api_proxy-main'
min_duration = 5

query = f"fields @timestamp, @message \
    | filter @logStream like 'application/nginx' \
    | filter request_time > {min_duration} \
    | sort @timestamp desc \
    | limit 100"

print (f'Querying log group: {log_group}. Looking for slow requests... (> {min_duration}s)')
data = cloudwatch_query(logs_client, log_group, query, start_time=days_ago(3))
parse_json_messages(data)
print (json.dumps(data, indent=4))

Querying log group: projects-main. Looking for slow requests... (> 5s)
Query still running...
Query still running...
Query still running...
Query completed.
records_scanned: 176881464
records_matched: 21416
[
    {
        "@timestamp": "2024-01-25 09:16:33.138",
        "@message": {
            "time_local": "25/Jan/2024:09:16:33 +0000",
            "client": "10.185.254.11",
            "ACL_Cloudfront": "",
            "method": "GET",
            "scheme": "http",
            "host": "projects-eu.highbond.com",
            "request": "GET /api/v3/projects?org_id=4455&fields%5Bprojects%5D=custom_attributes%2Cattributes%2Ccollaborator_groups%2Ccollaborators%2Cname%2Cstate%2Cstatus%2Cplanned_start_date%2Cactual_start_date%2Cplanned_end_date%2Cactual_end_date%2Cplanned_milestone_date%2Cstart_date%2Ctarget_date%2Cactual_milestone_date&page%5Bsize%5D=100&page%5Bnumber%5D=MTI%3D HTTP/1.1",
            "request_length": "4125",
            "status": "200",
            "bytes_sent": "14130

In [ ]:
request_ids = collect_request_ids(data)

print (json.dumps(request_ids, indent=4))

# Collect All Requests and Aggregate on Total Duration

In [ ]:
log_group = 'projects-main'
#log_group =  'api_proxy-main'
#log_group = 'results-main'

result = cloudwatch_query(logs_client, log_group, query, start_time=days_ago(3))

# Use ThreadPoolExecutor to run the loop in parallel
with ThreadPoolExecutor(max_workers=8) as executor:
    # Map the function to your request_ids
    results = executor.map(process_request, request_ids[:24])

    # Extend the data list with results from the threads
    for request_id,result in results:
        data[request_id] = result

print (json.dumps(data, indent=4))